Bonus track
# Using Plumed 2 CVs as metrics
## Toni Giorgino
Institute of Neurosciences  
National Research Council of Italy

<div>
<img src="img/plumed-logo.png" style="width: 20%; float: right"/>
</div>

# Overview

Plumed 2 (www.plumed.org) is an open source library 
for free energy calculations in molecular systems. 
It provides a rich array of collective variables (CVs),
defined through an intuitive language, e.g.

    phi: TORSION ATOMS=5,7,9,15
    psi: TORSION ATOMS=7,9,15,17
    
Of several Plumed function, we are interested
in *evaluating* CVs on
existing simulations (`driver`). 

We can code HTMD metrics in Plumed2 syntax seamlessly.

# What's in for me?

Plumed [includes](https://plumed.github.io/doc-v2.3/user-doc/html/index.html)
very complex CVs, yet they are coded in an intuitive 
and concise syntax. (Especially true with PLUMED-GUI, see later). E.g.

* Coordination numbers `COORDINATION`
* Native contacts
* Amount of &alpha; and &beta; (anti/parallel) content
* S- and Z- path variables
* Virtual atoms, PCAs, dipoles, ...


# Set-up



In [ ]:
# %qtconsole
from htmd import *
# Being an experimental feature, we import it explicitly:
from htmd.projections.metricplumed2 import * 
htmd.projections.metricplumed2._getPlumedRoot() # Check

# CVs as Metrics - simplest

In [ ]:
metric = MetricPlumed2(['d1: DISTANCE ATOMS=1,200',
                        'd2: DISTANCE ATOMS=5,6'])

In [ ]:
mol = Molecule("3PTB")
metric.project(mol)

**Note** CVs were entered as strings; atoms are inconveniently selected as serials. Next slides will provide a solution.

# Object-oriented CV creation (1)

Instead of strings, we can pass `PlumedCV` objects. They are created by passing the following arguments, in order...

1. the collective variable (`DISTANCE`, `GYRATION`,...)
2. an unique label (autogenerated if omitted)
3. CV arguments, as Python named arguments (pass `True` for switches)

In [ ]:
gyr1 = PlumedCV("GYRATION", "rgyr", ATOMS="10-20", TYPE="RADIUS")
str(gyr1)

# We also have gyr1.label, gyr1.args etc

# Object-oriented CV creation (2)

*Atom groups* and *centers of masses* can also be generated symbolically and passed *in lieu* of atom indices or lists.

Arguments to `PlumedGroup` and `PlumedCOM` are, in order
1. the molecule corresponding to the system
2. the group label
3. an atom selection

Examples:


In [ ]:
m   =  Molecule("3PTB")
grp =  PlumedGroup(m,"grp","serial 10 to 20")
str(grp)

In [ ]:
gyr2 = PlumedCV("GYRATION", "rgyr2", 
                ATOMS=grp, TYPE="ASPHERICITY", NOPBC=True)
str(gyr2)

In [ ]:
protCA = PlumedCOM(m,"protCA","chain A and name CA")
lig    = PlumedCOM(m,"lig","resname BEN and noh")
dCAlig = PlumedCV("DISTANCE", "dist", ATOMS=[protCA,lig]) 

str(dCAlig)


# Help on functions

Use `genTemplate()` to see the arguments required by a specific CV. `optional=True` includes optional arguments.

In [ ]:
htmd.projections.metricplumed2.genTemplate("COORDINATION")

In [ ]:
htmd.projections.metricplumed2.genTemplate("COORDINATION",
                                           include_optional=True)

# PLUMED-GUI

CVs are many and sometimes non-trivial to write - e.g. secondary structure
content, RMSD, paths, native contacts and so on. 

VMD ships with a [PLUMED-GUI](http://dx.doi.org/10.1016/j.cpc.2013.11.019) for prototyping and evaluating
Plumed metrics - see in the *Extensions/Analysis* menu.

![plumed-gui](img/plumed-gui.png)


You can write symbolic expressions for atomselections, in square brackets. They
will be substituted when you "Export" the script. E.g.:

```
UNITS  LENGTH=A  ENERGY=kcal/mol  TIME=ps
protein: COM ATOMS=[chain A and name CA]
ligand:  COM ATOMS=[chain B and noh]
DISTANCE ATOMS=protein,ligand
```


# Citations

Please acknowledge your use of PLUMED2 by citing...
 * Tribello, G. A., Bonomi, M., Branduardi, D., Camilloni, C., & Bussi, G. (2014). PLUMED 2: New feathers for an old bird. Computer Physics Communications, 185(2), 604-613.
 
For PLUMED-GUI...
 * Giorgino T, PLUMED-GUI: An environment for the interactive development of molecular dynamics analysis and biasing scripts, Computer Physics Communications, 185, 3, 2014: 1109-1114 
